In [0]:
df = spark.table("book_reviews")   # table name is whatever you chose on upload
df.show(5)


+---+----------------+--------------------+------+--------------------+--------+
| id|          author|          book_title|rating|              review|reviewer|
+---+----------------+--------------------+------+--------------------+--------+
|  1|    Paulo Coelho|       The Alchemist|     5|A timeless tale o...|   User1|
|  2|     James Clear|       Atomic Habits|     4|Practical and mot...|   User2|
|  3|Robert C. Martin|          Clean Code|     5|Essential for eve...|   User3|
|  4|   George Orwell|                1984|     4|Dark dystopian cl...|   User4|
|  5|      Harper Lee|To Kill a Mocking...|     5|Powerful story on...|   User5|
+---+----------------+--------------------+------+--------------------+--------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col, when, length

# Clean and enrich data
silver_df = (
    df.dropDuplicates(["book_title", "reviewer"])
      .filter(col("review").isNotNull())
      .withColumn("rating", col("rating").cast("int"))
      .withColumn(
          "rating_category",
          when(col("rating") >= 4, "Positive")
          .when(col("rating") == 3, "Neutral")
          .otherwise("Negative")
      )
      .withColumn("review_length", length(col("review")))
)

# View cleaned data
silver_df.show(5)


+---+----------------+--------------------+------+--------------------+--------+---------------+-------------+
| id|          author|          book_title|rating|              review|reviewer|rating_category|review_length|
+---+----------------+--------------------+------+--------------------+--------+---------------+-------------+
|  1|    Paulo Coelho|       The Alchemist|     5|A timeless tale o...|   User1|       Positive|           36|
|  2|     James Clear|       Atomic Habits|     4|Practical and mot...|   User2|       Positive|           35|
|  3|Robert C. Martin|          Clean Code|     5|Essential for eve...|   User3|       Positive|           30|
|  4|   George Orwell|                1984|     4|Dark dystopian cl...|   User4|       Positive|           23|
|  5|      Harper Lee|To Kill a Mocking...|     5|Powerful story on...|   User5|       Positive|           35|
+---+----------------+--------------------+------+--------------------+--------+---------------+-------------+
o

In [0]:
df.printSchema()


root
 |-- id: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- book_title: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- reviewer: string (nullable = true)



In [0]:
from pyspark.sql.functions import avg, count

gold_df = (
    silver_df.groupBy("book_title", "author")
              .agg(
                  avg("rating").alias("avg_rating"),
                  count("*").alias("total_reviews")
              )
)

gold_df.show(10)


+--------------------+-----------------+----------+-------------+
|          book_title|           author|avg_rating|total_reviews|
+--------------------+-----------------+----------+-------------+
|       The Alchemist|     Paulo Coelho|       5.0|            1|
|       Atomic Habits|      James Clear|       4.0|            1|
|          Clean Code| Robert C. Martin|       5.0|            1|
|                1984|    George Orwell|       4.0|            1|
|To Kill a Mocking...|       Harper Lee|       5.0|            1|
|             Sapiens|Yuval Noah Harari|       4.0|            1|
|    The Lean Startup|        Eric Ries|       4.0|            1|
|                Dune|    Frank Herbert|       5.0|            1|
|            Educated|    Tara Westover|       5.0|            1|
|          The Hobbit|   J.R.R. Tolkien|       5.0|            1|
+--------------------+-----------------+----------+-------------+
only showing top 10 rows
